In [1]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic

In [2]:
import gradio as gr # oh yeah!

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBb


In [4]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [31]:
# A generic system message - no more snarky adversarial AIs!

system_message = "You are a search assistant that takes the user input prompt as the main topic of search for the top 10 companies that work on this topic.Also keep the URL of these 10 companies in a list called company_list that I can use for further analysis. "

In [32]:
# Let's wrap a call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [33]:
# This can reveal the "training cut off", or the most recent date in the training data

message_gpt("What is today's date?")

"Today's date is April 27, 2024."

In [34]:
# Let's create a call that streams back results
# If you'd like a refresher on Generators (the "yield" keyword),
# Please take a look at the Intermediate Python notebook in week1 folder.

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [35]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.


In [36]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [37]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.


In [38]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [39]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="What types of companies are you interested in:"), gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.


# Building a company brochure generator

Now you know how - it's simple!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you read the next few cells</h2>
            <span style="color:#900;">
                Try to do this yourself - go back to the company brochure in week1, day5 and add a Gradio UI to the end. Then come and look at the solution.
            </span>
        </td>
    </tr>
</table>

In [41]:
company_list = [
    "https://www.alterecofoods.com",
    "https://www.naturespath.com",
    "https://www.ancientharvest.com",
    "http://www.quinoacorp.com",
    "https://www.bobsredmill.com",
    "https://www.edenfoods.com",
    "https://www.quinuacereal.com",
    "https://www.wholefoodsmarket.com",
    "https://www.terrasoul.com",
    "https://www.thebeautifulbrinysea.com"
]

In [42]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [43]:

system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [44]:
def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [46]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.


In [ ]:

    def build_company_profile_prompt(companyName,url):
        """Build prompt for generating company profile."""
        prompt = f"""
        Please analyze this company information {companyName}, here is the landing page: {url} and create a detailed company profile:
        url
        Company Name: {self.company_info.get('name', 'Unknown')}
        Tagline: {self.company_info.get('tagline', '')}
        Mission: {self.company_info.get('mission', '')}
        Vision: {self.company_info.get('vision', '')}
        Values: {', '.join(self.company_info.get('values', []))}
        Unique Selling Points: {', '.join(self.company_info.get('unique_selling_points', []))}
        Founded: {self.company_info.get('founded', '')}
        Locations: {', '.join(self.company_info.get('locations', []))}
        Products/Services: {', '.join(self.company_info.get('products_services', []))}
        Industries Served: {', '.join(self.company_info.get('industries_served', []))}
        Key Phrases: {', '.join(self.company_info.get('key_phrases', []))}
        
        Based on this information, please create:
        1. A comprehensive company profile (2-3 paragraphs)
        2. A concise value proposition statement
        3. Identification of the company's core competencies
        4. A summary of the target market and ideal customer profile
        """
        
        return prompt
        
    def build_competitive_analysis_prompt(company_name):
        """Build prompt for generating competitive analysis."""
        # Base company info
        company_name = self.company_info.get('name', 'Unknown')
        
        prompt = f"""
        Please generate a comprehensive competitive analysis report for {company_name}.
        
        COMPANY INFORMATION:
        Company Name: {company_name}
        Tagline: {self.company_info.get('tagline', '')}
        Mission: {self.company_info.get('mission', '')}
        Products/Services: {', '.join(self.company_info.get('products_services', []))}
        Unique Selling Points: {', '.join(self.company_info.get('unique_selling_points', []))}
        Key Phrases: {', '.join(self.company_info.get('key_phrases', []))}
        
        COMPETITORS:
        """
            
        prompt += f"""
        Based on this information, please create a professional competitive analysis report with the following sections:
        
        1. Executive Summary
           - Brief overview of {company_name}'s market position
           - Summary of key competitive advantages and challenges
        
        2. Industry Overview
           - Analysis of the current market landscape
           - Key industry trends and growth projections
           - Regulatory or technological factors affecting the industry
        
        3. Competitor Profiles
           - Detailed analysis of each competitor's strengths and weaknesses
           - Comparative assessment of product/service offerings
           - Market positioning and target audience analysis
        
        4. Competitive Advantage Analysis
           - {company_name}'s unique value proposition in the marketplace
           - Areas where {company_name} outperforms competitors
           - Areas where competitors have an edge over {company_name}
        
        5. SWOT Analysis
           - Detailed Strengths, Weaknesses, Opportunities, and Threats analysis for {company_name}
           - How these factors compare to key competitors
        
        6. Market Positioning Map
           - Describe how {company_name} and competitors are positioned on key attributes
           - Identify potential positioning opportunities in the market
        
        7. Strategic Recommendations
           - Areas where {company_name} should focus to strengthen competitive position
           - Potential threats to address and opportunities to pursue
           - Short-term and long-term strategic priorities
        
        Please be specific, data-driven, and insightful in your analysis. Include concrete examples wherever possible.
        """
        
        return prompt
    
    def build_swot_analysis_prompt(self):
        """Build prompt specifically for SWOT analysis."""
        company_name = self.company_info.get('name', 'Unknown')
        
        prompt = f"""
        Please conduct a detailed SWOT (Strengths, Weaknesses, Opportunities, Threats) analysis for {company_name} based on the following information:
        
        COMPANY INFORMATION:
        Company Name: {company_name}
        Tagline: {self.company_info.get('tagline', '')}
        Mission: {self.company_info.get('mission', '')}
        Products/Services: {', '.join(self.company_info.get('products_services', []))}
        Unique Selling Points: {', '.join(self.company_info.get('unique_selling_points', []))}
        Key Phrases: {', '.join(self.company_info.get('key_phrases', []))}
        
        COMPETITORS:
        """
        
        # Add competitor information
        for i, competitor in enumerate(self.competitors_info, 1):
            prompt += f"""
            Competitor {i}: {competitor.get('name', f'Competitor {i}')}
            URL: {competitor.get('url', '')}
            Description: {competitor.get('description', '')}
            Products/Services: {', '.join(competitor.get('products_services', []))}
            """
        
        prompt += f"""
        For each SWOT category, please provide:
        1. A bulleted list of at least 5 factors
        2. A brief explanation of each factor's significance
        3. A "Priority Level" rating (High/Medium/Low) for each factor based on its strategic importance
        
        For the Strengths and Weaknesses analysis, focus on internal factors like:
        - Unique technological capabilities
        - Brand reputation and recognition
        - Quality of products/services
        - Customer service and support
        - Financial resources and stability
        - Operational efficiency
        - Talent and expertise
        - Marketing effectiveness
        
        For the Opportunities and Threats analysis, focus on external factors like:
        - Market trends and growth potential
        - Technological developments
        - Regulatory changes
        - Competitive landscape shifts
        - Economic factors
        - Social/demographic changes
        - Emerging markets or customer segments
        
        Please make the analysis as specific and actionable as possible, avoiding generic statements.
        """
        
        return prompt

In [53]:
 def build_company_profile_prompt(companyName,url,model):
        """Build prompt for generating company profile."""
        prompt = f"""
        Please analyze this company information {companyName}, here is the landing page: {url} and create a detailed company profile for each of the following sections:
        url
        Company Name: 
        Tagline: 
        Mission: 
        Vision: 
        Values: 
        Unique Selling Points: 
        Founded: 
        Locations: 
        Products/Services: 
        Industries Served: 
        Key Phrases:
        
        Based on this information, please create:
        1. A comprehensive company profile (2-3 paragraphs)
        2. A concise value proposition statement
        3. Identification of the company's core competencies
        4. A summary of the target market and ideal customer profile

        Write it all in markdown
        """

        prompt += Website(url).get_contents()
        if model=="GPT":
            result = stream_gpt(prompt)
        elif model=="Claude":
            result = stream_claude(prompt)
        else:
            raise ValueError("Unknown model")
        yield from result

In [54]:
view = gr.Interface(
    fn=build_company_profile_prompt,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="CompanyInformation:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7885

To create a public link, set `share=True` in `launch()`.


In [63]:
 def build_competitor_prompt(companyName,model):
        """Build prompt for generating company profile."""
        prompt = f"""
        For the given industry mentioned in user prompt {companyName}, take top 10 competitors in this field.         

        List each of the 10 competitors with the following information: 
        Company Name: 
        Tagline: 
        Mission: 
        Products/Services: 
        Unique Selling Points: 
        Key Phrases:

        I want to create a new company called "NewCompanyByZahra"
        
        Next create a professional competitive analysis report with the following sections:
        
        1. Executive Summary
           - Brief overview of NewCompanyByZahra's market position
           - Summary of key competitive advantages and challenges
        
        2. Industry Overview
           - Analysis of the current market landscape
           - Key industry trends and growth projections
           - Regulatory or technological factors affecting the industry
        
        3. Competitor Profiles
           - Detailed analysis of each competitor's strengths and weaknesses
           - Comparative assessment of product/service offerings
           - Market positioning and target audience analysis
        
        4. Competitive Advantage Analysis
           - NewCompanyByZahra's unique value proposition in the marketplace
           - Areas where NewCompanyByZahra outperforms 10 competitors
           - Areas where  10 competitors have an edge over NewCompanyByZahra
        
        5. SWOT Analysis
           - Detailed Strengths, Weaknesses, Opportunities, and Threats analysis for the new company
           - How these factors compare to key competitors
        
        6. Market Positioning Map
           - Describe how the 10 competitors are positioned on key attributes
           - Identify potential positioning opportunities in the market
        
        7. Strategic Recommendations
           - Areas where NewCompanyByZahra should focus to strengthen competitive position
           - Potential threats to address and opportunities to pursue
           - Short-term and long-term strategic priorities
        
        Please be specific, data-driven, and insightful in your analysis. Include concrete examples wherever possible.    

        Write it all in markdown
        """
        if model=="GPT":
            result = stream_gpt(prompt)
        elif model=="Claude":
            result = stream_claude(prompt)
        else:
            raise ValueError("Unknown model")
        yield from result

In [60]:
view = gr.Interface(
    fn=build_competitor_prompt,
    inputs=[
        gr.Textbox(label="What sort of company do you want to look into:"),
        gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Competitor analysis:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7886

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 2136, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 1674, in call_function
    prediction = await utils.async_iteration(iterator)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/utils.py", line 728, in async_iteration
    return await anext(it